In [2]:
from lines_generator import dataGenerator
import numpy as np
import keras
from keras import Sequential
from keras.layers import Dense,Conv2D, MaxPooling2D, Flatten

Using TensorFlow backend.


In [3]:
patche_size = 17
input_dim = patche_size**2
num_classes = 2

## ANN Train
Generator

In [10]:
gen_train = dataGenerator(50,'train',prob_peaks=0.2,prob_near_peaks=0.8)
gen_test = dataGenerator(10,'test',prob_peaks=0.2,prob_near_peaks=0.8)

__activations:__ = relu - softmax - sigmoid

In [11]:
# create model
model = Sequential()
model.add(Dense(350, input_dim=input_dim, activation='sigmoid'))
model.add(Dense(250, input_dim=input_dim, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

https://keras.io/models/model/ $\ \ \ $ https://keras.io/optimizers/ $\ \ \ $ https://keras.io/losses/  <br>
__optimizer:__ SGD - RMSprop - Adagrad - Adadelta - Adam - Adamax - Nadam - TFOptimizer. <br>
__loss:__ mean_squared_error - mean_absolute_error - mean_absolute_percentage_error - mean_squared_logarithmic_error - squared_hinge - hinge - categorical_hinge - logcosh - categorical_crossentropy - sparse_categorical_crossentropy - binary_crossentropy - kullback_leibler_divergence - poisson - cosine_proximity.

In [12]:
model.compile(optimizer='Adamax',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [13]:
model.fit_generator(gen_train, samples_per_epoch=64, nb_epoch=50)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=64, epochs=50)`
  """Entry point for launching an IPython kernel.


Epoch 1/50
64/64 [==============================] - 1s 14ms/step - loss: 0.4085 - acc: 0.8115
Epoch 2/50
64/64 [==============================] - 0s 5ms/step - loss: 0.2898 - acc: 0.8555
Epoch 3/50
64/64 [==============================] - 0s 5ms/step - loss: 0.2275 - acc: 0.8975
Epoch 4/50
64/64 [==============================] - 0s 5ms/step - loss: 0.2123 - acc: 0.9072
Epoch 5/50
64/64 [==============================] - 0s 5ms/step - loss: 0.2389 - acc: 0.8838
Epoch 6/50
64/64 [==============================] - 0s 5ms/step - loss: 0.1889 - acc: 0.9209
Epoch 7/50
64/64 [==============================] - 0s 5ms/step - loss: 0.1889 - acc: 0.9189
Epoch 8/50
64/64 [==============================] - 0s 5ms/step - loss: 0.2060 - acc: 0.9102
Epoch 9/50
64/64 [==============================] - 0s 7ms/step - loss: 0.1799 - acc: 0.9229
Epoch 10/50
64/64 [==============================] - 0s 5ms/step - loss: 0.1717 - acc: 0.9219
Epoch 11/50
64/64 [==============================] - 0s 7ms/step - l

In [14]:
score = model.evaluate_generator(gen_test,steps=1024)
score

[0.17585245807686078, 0.9276123046875]

In [15]:
pred_test = model.predict_generator(gen_test,steps=128,verbose=1)

128/128 [==============================] - 0s 2ms/step


In [16]:
pred_test

array([[2.7262050e-04, 9.9972731e-01],
       [8.0253327e-01, 1.9746673e-01],
       [6.6005610e-05, 9.9993396e-01],
       ...,
       [4.0374314e-03, 9.9596256e-01],
       [2.5628006e-01, 7.4372000e-01],
       [8.7177596e-04, 9.9912828e-01]], dtype=float32)

## Save

In [17]:
model.save('models/model4.h5')

## CNN Train 
Generator

In [4]:
gen_train = dataGenerator(100,'train',prob_peaks=0.2,prob_near_peaks=0.8,reshape=False,expand_dims=True)
gen_test = dataGenerator(20,'test',prob_peaks=0.2,prob_near_peaks=0.8,reshape=False,expand_dims=True)

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(patche_size,patche_size,1)))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='Adamax',#keras.optimizers.SGD(lr=0.01),
              metrics=['accuracy'])

In [7]:
model.fit_generator(gen_train, samples_per_epoch=1024, nb_epoch=10)

/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """Entry point for launching an IPython kernel.
/home/cordeiro/env/py3im/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=1024, epochs=10)`
  """Entry point for launching an IPython kernel.


Epoch 1/10
1024/1024 [==============================] - 31s 30ms/step - loss: 0.1765 - acc: 0.9223
Epoch 2/10
1024/1024 [==============================] - 7s 7ms/step - loss: 0.1128 - acc: 0.9545
Epoch 3/10
1024/1024 [==============================] - 7s 7ms/step - loss: 0.0851 - acc: 0.9693
Epoch 4/10
1024/1024 [==============================] - 7s 7ms/step - loss: 0.0813 - acc: 0.9714
Epoch 5/10
1024/1024 [==============================] - 8s 7ms/step - loss: 0.0701 - acc: 0.9752
Epoch 6/10
1024/1024 [==============================] - 8s 7ms/step - loss: 0.0667 - acc: 0.9763
Epoch 7/10
1024/1024 [==============================] - 8s 8ms/step - loss: 0.0549 - acc: 0.9821
Epoch 8/10
1024/1024 [==============================] - 9s 9ms/step - loss: 0.0538 - acc: 0.9813
Epoch 9/10
1024/1024 [==============================] - 8s 8ms/step - loss: 0.0536 - acc: 0.9823
Epoch 10/10
1024/1024 [==============================] - 8s 8ms/step - loss: 0.0473 - acc: 0.9847


In [9]:
score = model.evaluate_generator(gen_test,steps=1024)
score

[0.0818509245857797, 0.973388671875]

In [10]:
model.save('models/model5.h5')